In [ ]:
!gunzip *.jsonl.gz

gzip: *.jsonl.gz: No such file or directory


In [ ]:
import glob
import pandas as pd

In [ ]:
all_files = glob.glob('*.jsonl')

In [ ]:
all_files

['coronavirus-tweet-id-2020-06-26-05.jsonl',
 'coronavirus-tweet-id-2020-06-26-10.jsonl']

In [ ]:
li = []
for filename in all_files:
  df = pd.read_json(filename, lines=True)
  li.append(df)

In [ ]:
df = pd.concat(li, axis = 0, ignore_index = True)

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
#df.to_csv('feb1Tweets.csv')

In [ ]:
from datetime import datetime

In [ ]:
df = df[['created_at','full_text','place','lang']]

In [ ]:
bool_series = pd.notnull(df['place'])

In [ ]:
df = df[bool_series]

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
location = pd.DataFrame(df.place.tolist())

In [ ]:
df['area'] = location.full_name
df['country'] = location.country
df['country_code'] = location.country_code

In [ ]:
df.drop(columns=['place'],inplace=True)

In [ ]:
df.rename(columns={'created_at':"date","full_text":"tweet"},inplace=True)

In [ ]:
import re
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [ ]:
cleanTweets = []
for t in df.tweet:
    cleanTweets.append(tweet_cleaner(t))

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "https://t.co/UgQcjKpqSJ" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "https://t.co/IBFOqhQ4Fd" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "https://t.co/2o7d6r3tlZ" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful S

In [ ]:
df['tweet'] = cleanTweets

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
#df = df[df['tweet'].str.split().str.len().gt(5)]

In [ ]:
#df = df[df.lang.str.contains("en")]

In [ ]:
df

,date,tweet,lang,area,country,country_code
0,2020-06-26 04:59:56+00:00,nims says sll permission taken by patanjsli gr...,en,"Moradabad, India",India,IN
1,2020-06-26 04:59:57+00:00,you missed the point the point being so many s...,en,"Edinburgh, Scotland",United Kingdom,GB
2,2020-06-26 04:59:58+00:00,can i hope he dies from a bee infestation in h...,en,"Arvada, CO",United States,US
3,2020-06-26 04:59:59+00:00,kag,und,"Pasadena, CA",United States,US
4,2020-06-26 04:59:57+00:00,globalpolitics worldwide politics political nz...,und,"Auckland, New Zealand",New Zealand,NZ
...,...,...,...,...,...,...
2532,2020-06-26 10:59:44+00:00,tegen dat iedereen om welke reden dan ook zijn...,nl,"Aarschot, België",Belgium,BE
2533,2020-06-26 10:59:45+00:00,libdems r renshaw,und,"Worcester, England",United Kingdom,GB
2534,2020-06-26 10:59:46+00:00,wearing a mask and keeping distance as a sign ...,en,"Bexley, London",United Kingdom,GB
2535,2020-06-26 10:59:46+00:00,this is the other face of caring and thoughtfu...,en,"Edinburgh, Scotland",United Kingdom,GB


In [ ]:
df.to_csv('../juneTweets/juneTweets31.csv')

In [ ]:
df.lang.value_counts()

en     1620
es      217
und     178
in      130
pt       85
hi       69
tl       40
fr       37
de       28
nl       25
it       23
tr       22
ja       15
sv        8
et        5
th        4
ht        4
ru        3
ro        3
pl        3
ca        2
da        2
ta        2
ar        2
cs        2
no        1
iw        1
pa        1
lt        1
uk        1
ml        1
ur        1
mr        1
Name: lang, dtype: int64

In [ ]:
df[.query('lang=="en"').count()

date            1620
tweet           1620
lang            1620
area            1620
country         1620
country_code    1620
dtype: int64

In [ ]:
df.query('lang=="en" and country_code=="IN"')

,date,tweet,lang,area,country,country_code
0,2020-06-26 04:59:56+00:00,nims says sll permission taken by patanjsli gr...,en,"Moradabad, India",India,IN
5,2020-06-26 05:00:02+00:00,carpool mumbai jaipur lock down permission ava...,en,"Mumbai, India",India,IN
18,2020-06-26 05:00:36+00:00,word,en,"Raichur, India",India,IN
31,2020-06-26 05:01:30+00:00,csjmuagainststudentlife promoteallstudents ups...,en,"Kalyanpur Kanpur, Kanpur",India,IN
33,2020-06-26 05:01:33+00:00,agreed india love instagram photography mumbai...,en,"Mumbai, India",India,IN
...,...,...,...,...,...,...
2486,2020-06-26 10:57:13+00:00,another covid patients have recovered and are ...,en,"Cuttack, India",India,IN
2497,2020-06-26 10:57:39+00:00,thanks china and arvind kejriwal for gifting c...,en,"New Delhi, India",India,IN
2508,2020-06-26 10:58:10+00:00,early lockdown was done primarily to break cha...,en,"Bengaluru, India",India,IN
2510,2020-06-26 10:58:17+00:00,ggsipu cancel backlog and final year students ...,en,"New Delhi, India",India,IN


In [ ]:
#type(df.date.iloc[0])

In [ ]:
#df.sort_values(by='date')